
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>

---


# TD 9 : Théorie de l'information
___

### Description

Dans ce travail dirigé, vous aurez l'occasion de déterminer quels mots sont les plus discriminants pour la construction du filtre anti-spam du TD8.

L'exercice 1 vous permettra de programmer les fonctions utiles pour classer les mots en fonction de leur information mutuelle avec la catégorie du message (courriel ou pourriel). L'exercice 2 vous permettra d'estimer la loi conjointe et les lois marginales entre une variable explicative $X_j$ et $Y$ avec les messages électroniques obtenus. Dans l'exercice 3, vous identifierez les mots les plus discriminant pour classer les messages électroniques en courriel ou pourriel.

### Données

Les données exploitées dans ce TD correspondent à des messages électroniques authentiques d'un employé de la compagnie Enron. Vous pouvez télécharger le jeux de données à partir du site web du cours. Le fichier doit être décompressé dans un dossier nommé *data* du répertoire courant de votre calepin Jupyter. 

Ces données ont déjà été utilisées dans le TD8. 


In [88]:
using Statistics, DataFrames

## Préliminaires

Cette section permet de dénombrer le nombre de courriels et de pourriels contenant chacun des mots répertoriés. Les deux dictionnaires suivants sont retournés :
- ham_wordcounts
- spam_word_counts

In [44]:
"""
wordlisting : Cette fonction transforme un fichier texte en une liste de mots. Le nombre de fois que
              les mots apparaîssent ne sont pas répertorié.
Input: filename::String le chemin du fichier texte 
Output: Array{Sting} Liste des mots contenus dans le fichier texte
"""

function wordlisting(filename::String)
    
    f = read(filename, String)
    text = replace(f, r"[0123456789]" => "")
    words = split(text, r"\W+")
    filter!(x -> length(x) > 1, words)
    wordlist = unique(words)
    
end

"""
wordcounting : À partir d'un Array of Array contenant la liste des mots d'un ensemble de fichiers texte, la fonction 
               retourne le nombre de fois où chaque mot est présent.
Input: Un array correspondant à la liste des mots pour chacun des fichiers texte 
Output: Dictionnaire compilant le nombre de ligne dans lesquelles chacun des mots apparaît.
"""

function wordcounting(A::Array{Array{SubString{String},1},1})
   
    words = vcat(A...)

    wordcounts = Dict{String,Int64}()

    for word in words
        wordcounts[word]=get(wordcounts, word, 0) + 1
    end
    
    return wordcounts
    
end


# Récupération des noms de fichier de tous les hams
filesdir = "../Exercices/Ressources/enron1/ham/"
filename_ham = filesdir.*readdir(filesdir)

# Récupération des noms de fichier de tous les spams
filesdir = "../Exercices/Ressources/enron1/spam/"
filename_spam = filesdir.*readdir(filesdir)

ham_wordlist = wordlisting.(filename_ham)
ham_wordcounts = wordcounting(ham_wordlist)

spam_wordlist = wordlisting.(filename_spam)
spam_wordcounts = wordcounting(spam_wordlist);

## Exercice 1

Programmation des fonctions utiles en théorie de l'information.

### (a) Entropie 

Programmez une fonction permettant de calculer l'entropie d'une variable aléatoire catégorielle $Y$ ayant comme argument le vecteur de probabilités des résulats possibles de $Y$.

Testez votre fonction de la façon suivante :

`p = [1/4,1/4,1/4,1/4]`

`entropy(p)`

In [45]:
function log2_2(x)
    x > 0 ? log2(x) : 0
end
function entropy(p::Vector{<:Real})
    return -log2_2.(p)'*p
end
p = [1/4,1/4,1/4,1/4]
entropy(p)

2.0

### (b) Entropie croisée

Programmez une fonction permettant de calculer l'entropie croisée d'une variable aléatoire catégorielle $Y$ ayant comme argument sont vecteur de probabilités des résulats possibles et comme deuxième argument la second vecteur de probabité correspondant à une autre fonction de masse pour $Y.

Testez votre fonction de la façon suivante :

`p = [1/2,1/4,1/8,1/8]`

`q = [1/4,1/4,1/4,1/4]`

`crossentropy(p,q)`

In [46]:
function crossentropy(p::Vector{<:Real},q::Vector{<:Real})
    -log2.(q)'*p
end
p = [1/2,1/4,1/8,1/8]
q = [1/4,1/4,1/4,1/4]
crossentropy(p,q)

2.0

### (c) Divergence de Kullback-Leibler

Programmez une fonction permettant de calculer la divergence de Kullback-Leibler entre les deux fonctions de masse caractérisées par les vecteurs de probabilités $p$ et $q$.

Testez votre fonction de la façon suivante :

`p = [1/2,1/4,1/8,1/8]`

`q = [1/4,1/4,1/4,1/4]`

`crossentropy(p,q)`

In [47]:
function kullback(p::Vector{<:Real}, q::Vector{<:Real})
       -entropy(p)+crossentropy(p,q)
end
p = [1/2,1/4,1/8,1/8]
q = [1/4,1/4,1/4,1/4]
kullback(p,q)

0.25

## Exercice 2 

Estimation des lois conjointes et marginales des variables $X$ et $Y$ avec le jeu de données.

### (a) Soit le mot *enron*, construisez le tableau des fréquences observées de la variable $X$ et $Y$.

Ce tableau des fréquences observées est illustré à l'exemple 8 du Chapitre 9 des notes de cours. Servez vous des dictionnaires ham_wordcounts et spam_wordcounts.

In [48]:
mot = "enron"
freq = zeros(2,2)
# À COMPLÉTER
freq[2,1] = count(in.("enron", spam_wordlist))
freq[2,2] = count(in.("enron", ham_wordlist))
freq[1,1] = length(filename_spam) - freq[2,1]
freq[1,2] = length(filename_ham) - freq[2,2]
freq

2×2 Array{Float64,2}:
 1500.0  2210.0
    0.0  1462.0

### (b) Avec le tableau des fréquences observées, estimez la loi de probabilité conjointe de $X$ et $Y$.

Produisez un vecteur de probabilités représentant les 4 possibilités.

In [49]:
n = (length(filename_spam)+length(filename_ham))
prob_conj = freq./n

2×2 Array{Float64,2}:
 0.290023  0.427301
 0.0       0.282676

### (c) Avec le tableau des fréquences observées, estimez la loi de probabilité conjointe de $X$ et $Y$ estimée à l'aide du produit des lois marginales.

Produisez un vecteur de probabilités représentant les 4 possibilités.

In [50]:
prob_marg = freq*ones(2,2)*freq./(n^2)

2×2 Array{Float64,2}:
 0.208041   0.509283
 0.0819826  0.200693

### (d) Estimez l'information mutuelle entre $X$ et $Y$.

Utilisez la loi conjointe estimée et le produit des lois marginales estimées dans la divergence de Kullback-Leibler.

In [51]:
kullback(reshape(prob_conj, 4,1)[:,1], reshape(prob_marg, 4,1)[:,1])

0.17049542401082207

## Exercice 3

Identification des mots les plus discriminants.

### (a) En reprenant la méthode de l'exercice 2, calculez l'information mutuelle pour tous les mots contenus dans les messages.

In [74]:
function calc_score(mot)
    F = zeros(2,2)
    F[2,1] = count(in.(mot, spam_wordlist))
    F[2,2] = count(in.(mot, ham_wordlist))
    F[1,1] = length(filename_spam) - F[2,1]
    F[1,2] = length(filename_ham) - F[2,2]
    prob_conj = F./n
    prob_marg = F*ones(2,2)*F./(n^2)
    return kullback(reshape(prob_conj, 4,1)[:,1], reshape(prob_marg, 4,1)[:,1])
end

calc_score (generic function with 1 method)

In [94]:
ham_words = collect(keys(ham_wordcounts))
scores = Dict(words .=> calc_score.(ham_words))

UndefVarError: UndefVarError: filenames not defined

In [ ]:
spam_words = collect(keys(spam_wordcounts))
for spam_word in spam_words
    if ! (spam_word in words)
        scores[spam_word] = calc_score(spam_word)
    end
end

### (b) Selon l'information mutuelle, quels sont les 10 mots les plus dépendants de $Y$ ?

In [ ]:
df = DataFrame(word = collect(keys(scores)), score = collect(values(scores)))

In [93]:
sort!(df, [:score], rev = true);
df

,word,score
,String,Float64
1,Subject,NaN
2,enron,0.170495
3,cc,0.131203
4,hpl,0.121008
5,daren,0.112389
6,http,0.100842
7,gas,0.0989163
8,forwarded,0.0954772
9,pm,0.0934839
